In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import random
import math

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics 
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [ ]:
data = pd.read_csv('/content/drive/My Drive/Test/HW2/2/prison_dataset.csv', header = None, skiprows = 1)
labels = (pd.read_csv('/content/drive/My Drive/Test/HW2/2/prison_dataset.csv', header = None, nrows = 1)).to_numpy()
data_np = data.to_numpy()
dim = data_np.shape
rows_num = 4*dim[0]//5
indx = random.sample(range(dim[0]), rows_num)
train_data = []
test_data = np.delete(data_np, indx, axis = 0)
for i in range(rows_num):
  train_data.append(data_np[indx[i]])
train_data = np.array(train_data)

In [ ]:
def Entropy(input, labels):
  count = []
  for label in labels:
    count.append(np.count_nonzero(np.transpose(input) == label))
  E = 0
  dim = input.shape
  if dim[0] is 0:
    return E
  count[:] = [x / dim[0] for x in count]
  for c in count:
    if (c != 0):
      E -= c*math.log2(c)
  return E

In [ ]:
def Gain(S_entropy, p, entropies):
  g = (np.matmul(entropies, np.transpose(p))).diagonal()
  return np.argmin(g)

In [ ]:
def ID3(S, S_labels, features, features_labels):
  E = Entropy(S, S_labels)
  dim0 = len(features_labels)
  dim1 = 0
  for i in features_labels:
    dim1 = max(dim1, len(i))
  entropies = np.zeros((dim0, dim1))
  count = np.zeros((dim0, dim1))
  for i in range(dim0):
    for j in range(len(features_labels[i])):
      indxs = (np.where(features[:, i] == features_labels[i][j]))[0]
      entropies[i][j] = Entropy(np.take(S, indxs), S_labels)
      count[i][j] = len(indxs)
  indx = Gain(E, count, entropies)
  node_indxs = []
  for i in range(len(features_labels[indx])):
    node_indxs.append((np.where(features[:, indx] == features_labels[indx][i]))[0])
  return indx, node_indxs

In [ ]:
S_labels = np.unique(np.transpose(train_data[:, 10]))
features_labels = []
for i in range(10):
  features_labels.append(np.unique(np.transpose(train_data[:,i])))
indx1, node1 = ID3(train_data[:, 10], S_labels, train_data[:, 0:10], features_labels)

print(labels[0][indx1])
print(features_labels[indx1])
features_labels = np.delete(features_labels, indx1, axis = 0)
labels = np.delete(labels, indx1, axis = 1)

Fiscal Year Released
[2010 2013 2015]


In [ ]:
help2 = np.delete(train_data, indx1, axis = 1)
indx2 = [None]*len(node1)
node2 = [None]*len(node1)
for i in range(len(node1)):
  indx2[i], node2[i] = ID3(train_data[node1[i], 10], S_labels, help2[node1[i], 0:9], features_labels)

for i in range(len(indx2)):
  print(labels[0][indx2[i]])
  print(features_labels[indx2[i]])

Recidivism Reporting Year
[2013 2016 2018]
Age At Release
['<45' '>45']
Release Type
['Discharged End of Sentence' 'Parole']


In [ ]:
leaf_states = []
for i in range(3):
  train = train_data[node1[i], :]
  for j in range(len(node2[i])):
    o1 = np.where((train[node2[i][j], 10] == 0))
    o2 = np.where((train[node2[i][j], 10] == 1))
    if (len(o1[0]) > len(o2[0])):
      leaf_states.append(0)
    elif len(o2[0]) != 0:
      leaf_states.append(1)
print(leaf_states)

[1, 1, 1, 0, 0]


The state of every leaf is determined by cheking the majority of the data the has the specified value.

In [ ]:
def Test_ID3(test_data):
  confusion_matrix = np.zeros((2, 2))
  no_data = 0
  for i in range((test_data.shape)[0]):
    if ((test_data[i, 0] == 2010 and test_data[i, 1] == 2013) or (test_data[i, 0] == 2013)):
      if (test_data[i, 10] == 1):
        confusion_matrix[1, 1] += 1
      else:
        confusion_matrix[0, 1] += 1
    elif (test_data[i, 0] == 2015):
      if (test_data[i, 10] == 0):
        confusion_matrix[0, 0] += 1
      else:
        confusion_matrix[1, 0] += 1
    else:
      no_data += 1
  return confusion_matrix, no_data

In [ ]:
Confusion_Matrix, no_data = Test_ID3(test_data)
print('Confusion Matrix:\n', Confusion_Matrix)
accuracy = (Confusion_Matrix[0, 0]+Confusion_Matrix[1, 1])/(sum(sum(Confusion_Matrix)))
print('Accuracy is:  ', accuracy)

Confusion Matrix:
 [[1188.  208.]
 [ 650. 1039.]]
Accuracy is:   0.7218800648298217


In [ ]:
def Random_Forest(data, labels, features_labels, labels_name):
  p = 5
  indxs = random.sample(range(10), p)
  f = []
  for i in range(p):
    f.append(features_labels[indxs[i]])
  indx1, node1 = ID3(data[:, 10], labels, data[:, indxs], f)
  print(labels_name[0][indxs[indx1]])
  print(f[indx1])
  features_labels = np.delete(features_labels, indx1, axis = 0)
  labels_name = np.delete(labels_name, indx1, axis = 1)

  indx2 = [None]*len(node1)
  node2 = [None]*len(node1)
  for i in range(len(node1)):
    help2 = (np.delete(data, indx1, axis = 1))[node1[i]]
    indxs = random.sample(range(8), p)
    f = []
    for j in range(p):
      f.append(features_labels[indxs[j]])
    indx2[i], node2[i] = ID3(data[node1[i], 10], labels, help2[:, indxs], f)

  for i in range(len(indx2)):
    print(labels_name[0][indxs[indx2[i]]])
    print(f[indx2[i]])
  
  leaf_states = []
  for i in range(len(node1)):
    d = data[node1[i], :]
    for j in range(len(node2[i])):
      o1 = np.where((data[node2[i][j], 10] == 0))
      o2 = np.where((data[node2[i][j], 10] == 1))
      if (len(o1[0]) > len(o2[0])):
        leaf_states.append(0)
      elif len(o2[0]) != 0:
        leaf_states.append(1)
  print(leaf_states)

In [ ]:
train_data_rf = np.array_split(train_data, 4)

S_labels = np.unique(np.transpose(train_data[:, 10]))
features_labels = []
for i in range(10):
  features_labels.append(np.unique(np.transpose(train_data[:,i])))

In [ ]:
l = labels
f = features_labels
for i in range(4):
  Random_Forest(train_data_rf[i], S_labels, f, l)

Recidivism Reporting Year
[2013 2016 2018]
Convicting Offense Classification
['D Felony' 'Other Felony']
Age At Release
['<45' '>45']
Convicting Offense Classification
['D Felony' 'Other Felony']
[1, 1, 1, 1, 1, 1, 1]
Part of Target Population
['No' 'Yes']
Convicting Offense Subtype
['Other' 'Trafficking']
Recidivism Reporting Year
[2013 2016 2018]
[1, 1, 1, 1, 1, 1]
Convicting Offense Type
['Drug' 'Other' 'Violent']
Convicting Offense Type
['Drug' 'Other' 'Violent']
Convicting Offense Type
['Drug' 'Other' 'Violent']
Age At Release
['<45' '>45']
[1, 1, 1, 1, 1, 1, 1, 1]
Convicting Offense Type
['Drug' 'Other' 'Violent']
Convicting Offense Subtype
['Other' 'Trafficking']
Recidivism Reporting Year
[2013 2016 2018]
Recidivism Reporting Year
[2013 2016 2018]
[1, 1, 1, 1, 1, 1, 1, 1]


As you can see all of the leeves have the same label in all trees. I tried other ways too, but I think the problem is that variance of the given data is low and for getting better results we should train more trees and this code take so much time to handle the data.

In [ ]:
df = pd.read_csv('/content/drive/My Drive/Test/HW2/2/prison_dataset.csv')
df = df.apply(LabelEncoder().fit_transform)
X = df.drop('Recidivism - Return to Prison numeric', axis = 1)
y = df['Recidivism - Return to Prison numeric']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)
random_forest = RandomForestClassifier(n_estimators=30, max_depth=3, random_state=0)
random_forest.fit(X_train, y_train)
y_predict = random_forest.predict(X_test)

print('Accuracy is:\n', accuracy_score(y_test, y_predict))
pd.DataFrame(confusion_matrix(y_test, y_predict),columns=['Predicted 0', 'Predicted 1'],index=['True 0', 'True 1'])

Accuracy is:
 0.7277147487844409


,Predicted 0,Predicted 1
True 0,1084,217
True 1,623,1161


As we can see in this aproach we get bettter accuracy, because in this part we used the libraris of Python and they are so fast. So by training more trees we can get better results. 